In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pymysql
import time

conn = pymysql.connect(
    host="localhost",
    user="root",
    password="dnwhdgus9#",
    charset="utf8mb4",
    autocommit=True
)
cursor = conn.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS playground CHARACTER SET utf8mb4 COLLATE utf8mb4_unicode_ci;")
cursor.execute("USE playground;")

cursor.execute("""
CREATE TABLE IF NOT EXISTS books (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255),
    price_gbp DECIMAL(10,2),
    stock VARCHAR(255),
    rating INT,
    category VARCHAR(100),
    product_url VARCHAR(255) UNIQUE
);
""")

service = Service(ChromeDriverManager().install())

options = Options()
options.add_argument("--window-size=1920x1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")

driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)

base_url = "https://books.toscrape.com/"
driver.get(base_url)

rating_map = {"One": 1, "Two": 2, "Three": 3, "Four": 4, "Five": 5}

categories = driver.find_elements(By.CSS_SELECTOR, "div.side_categories ul li ul li a")
print(f"총 {len(categories)}개 카테고리")

for cat_idx in range(len(categories)):
    categories = driver.find_elements(By.CSS_SELECTOR, "div.side_categories ul li ul li a")
    cat = categories[cat_idx]
    cat_name = cat.text.strip()
    print(f"\n\n===== {cat_idx+1}. 카테고리 : {cat_name} =====")

    cat.click()
    time.sleep(1)

    page_num = 1
    while True:
        print(f"\n--- {cat_name} {page_num} 페이지 ---")

        titles = driver.find_elements(By.CSS_SELECTOR, "article.product_pod > h3")
        price_gbps = driver.find_elements(By.CSS_SELECTOR, "p.price_color")
        stocks = driver.find_elements(By.CSS_SELECTOR, "p.instock.availability")
        ratings = driver.find_elements(By.CSS_SELECTOR, "p.star-rating")
        product_links = driver.find_elements(By.CSS_SELECTOR, "article.product_pod h3 a")

        for i in range(len(titles)):
            rating_text = [c for c in ratings[i].get_attribute("class").split() if c != "star-rating"][0] if i < len(ratings) else None
            rating_number = rating_map.get(rating_text, 0) if rating_text else None

            title = titles[i].text if i < len(titles) else None
            price = float(price_gbps[i].text.replace("£", "")) if i < len(price_gbps) else None
            stock = stocks[i].text.strip() if i < len(stocks) else None
            url = product_links[i].get_attribute("href") if i < len(product_links) else None

            sql = """
            INSERT IGNORE INTO books (title, price_gbp, stock, rating, category, product_url)
            VALUES (%s, %s, %s, %s, %s, %s)
            """
            cursor.execute(sql, (title, price, stock, rating_number, cat_name, url))
            print(f"저장됨: {title}")

        try:
            next_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "li.next > a")))
            next_btn.click()
            time.sleep(2)
            page_num += 1
        except:
            print(f"{cat_name} 카테고리 마지막 페이지 도달")
            break

    driver.get(base_url)
    time.sleep(1)

driver.quit()
cursor.close()
conn.close()

총 50개 카테고리


===== 1. 카테고리 클릭: Travel =====

--- Travel 1 페이지 ---
저장됨: It's Only the Himalayas
저장됨: Full Moon over Noah’s ...
저장됨: See America: A Celebration ...
저장됨: Vagabonding: An Uncommon Guide ...
저장됨: Under the Tuscan Sun
저장됨: A Summer In Europe
저장됨: The Great Railway Bazaar
저장됨: A Year in Provence ...
저장됨: The Road to Little ...
저장됨: Neither Here nor There: ...
저장됨: 1,000 Places to See ...
Travel 카테고리 마지막 페이지 도달


===== 2. 카테고리 클릭: Mystery =====

--- Mystery 1 페이지 ---
저장됨: Sharp Objects
저장됨: In a Dark, Dark ...
저장됨: The Past Never Ends
저장됨: A Murder in Time
저장됨: The Murder of Roger ...
저장됨: The Last Mile (Amos ...
저장됨: That Darkness (Gardiner and ...
저장됨: Tastes Like Fear (DI ...
저장됨: A Time of Torment ...
저장됨: A Study in Scarlet ...
저장됨: Poisonous (Max Revere Novels ...
저장됨: Murder at the 42nd ...
저장됨: Most Wanted
저장됨: Hide Away (Eve Duncan ...
저장됨: Boar Island (Anna Pigeon ...
저장됨: The Widow
저장됨: Playing with Fire
저장됨: What Happened on Beale ...
저장됨: The Bachelor Girl's Guide 